<a href="https://colab.research.google.com/github/Romema1/Romema1/blob/main/Answer%20Assignment%203%20ISE%20571_PartII_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Answer Assignment 3 part II problem 1

In [ ]:
!pip install pyswarms

In [ ]:
import numpy as np
import pyswarms as ps

# ----- Problem Data -----

# Full-time staff rates (m = 5)
full_time_rates = np.array([30, 35, 40, 45, 50])
# Overtime rates (1.5x)
overtime_rates = 1.5 * full_time_rates  # [45, 52.5, 60, 67.5, 75]

# Part-time staff rates (n = 5)
part_time_rates = np.array([25, 27, 29, 31, 33])

# Number of staff groups
m = len(full_time_rates)
n = len(part_time_rates)

# Total dimension of decision vector:
# 5 full-time + 5 overtime + 5 part-time = 15
dim = m + m + n

# Staffing constraints:
min_full_time = 4   # At least 4 full-time staff during peak hours (count as sum of hours; here we assume hours are in some unit)
min_part_time = 2   # At least 2 part-time staff during non-peak hours

# To incorporate constraints in a continuous optimization via PSO, we use a penalty method.
# We assume that if the total scheduled "hours" are less than required, we add a penalty.
# (In a real scheduling problem, you might have integer decisions or binary selection; here we assume hours as continuous variables.)

# ----- Fitness Function Definition -----
def fitness_function(x):
    """
    x is a 2D array with shape (n_particles, dim)
    x has 15 components:
      - x[0:5] are full-time peak hours,
      - x[5:10] are full-time overtime hours,
      - x[10:15] are part-time non-peak hours.
    We compute total cost and add penalties if constraints are violated.
    """
    # Compute cost for each particle
    # Cost for full-time peak hours
    cost_full_time = np.dot(x[:, 0:m], full_time_rates)
    # Cost for overtime
    cost_overtime = np.dot(x[:, m:2*m], overtime_rates)
    # Cost for part-time
    cost_part_time = np.dot(x[:, 2*m:2*m+n], part_time_rates)

    total_cost = cost_full_time + cost_overtime + cost_part_time

    # Constraint penalties:
    # Constraint 1: Sum of full-time (peak + overtime) >= 4
    sum_full = np.sum(x[:, 0:m] + x[:, m:2*m], axis=1)
    # Constraint 2: Sum of part-time hours >= 2
    sum_part = np.sum(x[:, 2*m:2*m+n], axis=1)

    # Penalty factors (set large to discourage infeasible solutions)
    penalty_factor = 1e6

    penalty = np.zeros(x.shape[0])
    # If sum_full < min_full_time, add penalty proportional to the deficit squared.
    penalty += penalty_factor * np.maximum(0, (min_full_time - sum_full))**2
    # If sum_part < min_part_time, add penalty.
    penalty += penalty_factor * np.maximum(0, (min_part_time - sum_part))**2

    # Total fitness: we want to minimize cost, so fitness value is total_cost + penalty.
    # PSO in PySwarms minimizes the fitness function.
    fitness = total_cost + penalty
    return fitness

# ----- PSO Setup -----

# Define bounds for each decision variable.
# We need non-negative hours. Assume an upper bound for hours, say 10 hours per period (adjust as needed).
lb = np.zeros(dim)
ub = 10 * np.ones(dim)
bounds = (lb, ub)

# PSO parameters
options = {'w': 0.9, 'c1': 0.5, 'c2': 0.3}
swarm_size = 50
iterations = 1000

# ----- Run PSO using PySwarms -----

optimizer = ps.single.GlobalBestPSO(n_particles=swarm_size, dimensions=dim, options=options, bounds=bounds)
best_cost, best_pos = optimizer.optimize(fitness_function, iters=iterations, verbose=True)

# ----- Display the Results -----
print("#"*30)
print("Optimal Workforce Schedule (Decision Variables):")
print(np.round(best_pos, 3))
print("\nMinimum Total Labor Cost (with penalties if any):")
print(np.round(best_cost, 2))

# Interpret the result:
print("\nInterpretation:")
print("Full-time peak hours (first 5 entries):", np.round(best_pos[0:m], 3))
print("Full-time overtime hours (next 5 entries):", np.round(best_pos[m:2*m], 3))
print("Part-time non-peak hours (last 5 entries):", np.round(best_pos[2*m:2*m+n], 3))
print("\nStaffing sums:")
print("Total full-time (peak + overtime):", np.round(np.sum(best_pos[0:m] + best_pos[m:2*m]), 3))
print("Total part-time:", np.round(np.sum(best_pos[2*m:2*m+n]), 3))


2025-04-01 03:50:45,506 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'w': 0.9, 'c1': 0.5, 'c2': 0.3}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=726
2025-04-01 03:50:47,515 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 725.5008447068847, best pos: [1.23419156 0.53717537 0.91994747 1.12904433 0.80400712 3.05492025
 0.79923044 1.48200482 0.02369437 0.42188184 1.24217783 2.12811876
 2.42503475 1.91759365 0.66641939]


##############################
Optimal Workforce Schedule (Decision Variables):
[1.234 0.537 0.92  1.129 0.804 3.055 0.799 1.482 0.024 0.422 1.242 2.128
 2.425 1.918 0.666]

Minimum Total Labor Cost (with penalties if any):
725.5

Interpretation:
Full-time peak hours (first 5 entries): [1.234 0.537 0.92  1.129 0.804]
Full-time overtime hours (next 5 entries): [3.055 0.799 1.482 0.024 0.422]
Part-time non-peak hours (last 5 entries): [1.242 2.128 2.425 1.918 0.666]

Staffing sums:
Total full-time (peak + overtime): 10.406
Total part-time: 8.379


In [ ]:
# THE OPTIMAL SOLUTION FOR Q1

In [ ]:
import pulp as pl

# ----------------------------
# Problem Data and Parameters
# ----------------------------
# Full-time staff rates (for m = 5 full-time staff)
full_time_rates = [30, 35, 40, 45, 50]

# Overtime rates are 1.5× the full-time rates:
overtime_rates = [1.5 * rate for rate in full_time_rates]  # [45, 52.5, 60, 67.5, 75]

# Part-time staff rates (for n = 5 part-time staff)
part_time_rates = [25, 27, 29, 31, 33]

# Number of staff members
m = 5  # full-time staff
n = 5  # part-time staff

# Staffing constraints:
min_full_time = 4   # Total full-time (peak + overtime) hours must be at least 4
min_part_time = 2   # Total part-time hours must be at least 2

# ----------------------------
# MILP Model Formulation
# ----------------------------
# Create an LP minimization problem
prob = pl.LpProblem("Workforce_Scheduling", pl.LpMinimize)

# Decision Variables:
# x_i: number of peak hours for full-time staff i, i = 1,...,m
x = {i: pl.LpVariable(f"x_{i}", lowBound=0, cat="Continuous") for i in range(1, m+1)}

# x_io: number of overtime hours for full-time staff i, i = 1,...,m
x_io = {i: pl.LpVariable(f"x_io_{i}", lowBound=0, cat="Continuous") for i in range(1, m+1)}

# x_jp: number of hours for part-time staff j (non-peak), j = 1,...,n
x_jp = {j: pl.LpVariable(f"x_jp_{j}", lowBound=0, cat="Continuous") for j in range(1, n+1)}

# Objective Function:
# Minimize total labor cost:
#   total_cost = sum_{i=1}^{m} (c_i * x_i) + sum_{i=1}^{m} (c_io * x_io) + sum_{j=1}^{n} (c_jp * x_jp)
total_cost = (pl.lpSum([full_time_rates[i-1] * x[i] for i in range(1, m+1)]) +
              pl.lpSum([overtime_rates[i-1] * x_io[i] for i in range(1, m+1)]) +
              pl.lpSum([part_time_rates[j-1] * x_jp[j] for j in range(1, n+1)]))

prob += total_cost, "Total_Labor_Cost"

# Constraints:
# Peak-hour staffing requirement:
#   sum_{i=1}^{m} (x_i + x_io) >= 4
prob += pl.lpSum([x[i] + x_io[i] for i in range(1, m+1)]) >= min_full_time, "Peak_Staffing_Requirement"

# Non-peak-hour staffing requirement:
#   sum_{j=1}^{n} x_jp >= 2
prob += pl.lpSum([x_jp[j] for j in range(1, n+1)]) >= min_part_time, "NonPeak_Staffing_Requirement"

# ----------------------------
# Solve the MILP model using PuLP's default solver (CBC)
# ----------------------------
prob.solve()

# ----------------------------
# Display the Optimal Solution
# ----------------------------
print("Status:", pl.LpStatus[prob.status])
print("Optimal Workforce Schedule:")

print("\nFull-time staff (peak hours):")
for i in range(1, m+1):
    print(f"  Staff {i}: {x[i].varValue} hours")

print("\nFull-time staff (overtime hours):")
for i in range(1, m+1):
    print(f"  Staff {i}: {x_io[i].varValue} hours")

print("\nPart-time staff (non-peak hours):")
for j in range(1, n+1):
    print(f"  Staff {j}: {x_jp[j].varValue} hours")

print("\nMinimum Total Labor Cost:", pl.value(prob.objective))


Status: Optimal
Optimal Workforce Schedule:

Full-time staff (peak hours):
  Staff 1: 4.0 hours
  Staff 2: 0.0 hours
  Staff 3: 0.0 hours
  Staff 4: 0.0 hours
  Staff 5: 0.0 hours

Full-time staff (overtime hours):
  Staff 1: 0.0 hours
  Staff 2: 0.0 hours
  Staff 3: 0.0 hours
  Staff 4: 0.0 hours
  Staff 5: 0.0 hours

Part-time staff (non-peak hours):
  Staff 1: 2.0 hours
  Staff 2: 0.0 hours
  Staff 3: 0.0 hours
  Staff 4: 0.0 hours
  Staff 5: 0.0 hours

Minimum Total Labor Cost: 170.0
